In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

from requests import get 

url='https://www.indeed.fr/emplois?q=développeur%2C+data+scientist%2C+data+analyst%2C+business+intelligence&l=France'
response = get(url)
print(response.text[:500])

soup=BeautifulSoup(response.text,'html.parser')
type(soup)

<!DOCTYPE html>
<html lang="fr" dir="ltr">
<head>
<meta http-equiv="content-type" content="text/html;charset=UTF-8">
<script type="text/javascript">

(function ( tk ) { if ( tk && document.images ) { var s="/", q="?", a="&", e="="; (new Image()).src = s+"rpc"+s+"log"+q+"a"+e+"jslat"+a+"tk"+e+tk; } })('1e8orgm9f99oh800');
</script>
<script type="text/javascript" src="/s/870cbe7/fr_FR.js"></script>
<link href="/s/b512638/jobsearch_all.css" rel="stylesheet" type="text/css">
<link rel="alternate" ty


bs4.BeautifulSoup

In [2]:

dflocation = pd.DataFrame(columns=["location"])
dfcompany = pd.DataFrame(columns=["company"])
dfjob_title = pd.DataFrame(columns=["job_title"])
dfsalary = pd.DataFrame(columns=["salary"])
dfsummary= pd.DataFrame(columns=["summary"])
cities = ['Paris','Nancy','Lyon','Bordeaux','Metz','Toulouse','Marseille','Lille','Strasbourg','Nice']
df = pd.DataFrame(columns=["location", 'company', 'job_title', 'salary','summary'])
max_results_per_city = 200


In [3]:
def extract_job_title_from_result(soup):
    jobs = []

    for city in cities:
        for start in range(0, max_results_per_city, 10):
            page=requests.get('https://www.indeed.fr/emplois?q=développeur%2C+data+scientist%2C+data+analyst%2C+business+intelligence&l=' + str(city) + '&start=' + str(start))
            time.sleep(1)
            soup = BeautifulSoup(page.text,"lxml",from_encoding="utf-8")
            for div in soup.find_all(name="div", attrs={"class":"row"}):
                for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
                    jobs.append(a["title"])
    return(jobs)


In [5]:

titre=extract_job_title_from_result(soup)


In [6]:
def extract_company_from_result(soup):
    companies = []
   
    for city in cities:
        for start in range(0, max_results_per_city, 10):
            page=requests.get('https://www.indeed.fr/emplois?q=développeur%2C+data+scientist%2C+data+analyst%2C+business+intelligence&l=' + str(city) + '&start=' + str(start))
            time.sleep(1)
            soup = BeautifulSoup(page.text,"lxml",from_encoding="utf-8")       
            for div in soup.find_all(name="div", attrs={"class":"row"}):
                company = div.find_all(name="span", attrs={"class":"company"})
                if len(company) > 0:
                    for b in company:     
                        companies.append(b.text.strip())   
                else:   
                    sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
                    for span in sec_try: 
                          companies.append(span.text.strip())
    return(companies)


In [7]:
company = extract_company_from_result(soup)

In [8]:
def extract_location_from_result(soup):
    locations = [] 
   
    for city in cities:
        for start in range(0, max_results_per_city, 10):
            page=requests.get('https://www.indeed.fr/emplois?q=développeur%2C+data+scientist%2C+data+analyst%2C+business+intelligence&l=' + str(city) + '&start=' + str(start))
            time.sleep(1)
            soup = BeautifulSoup(page.text,"lxml",from_encoding="utf-8")       
            loc = soup.findAll('div', attrs={'class': 'location accessible-contrast-color-location'})   
            for div in soup.find_all(name="div", attrs={'class': 'location accessible-contrast-color-location'}):      
                locations.append(div.text)  
    return(locations)
#extract_location_from_result(soup)

In [9]:
location=extract_location_from_result(soup)

In [10]:
def extract_salary_from_result(soup):
    salaries = [] 
   
    for city in cities:
        for start in range(0, max_results_per_city, 10):
            page=requests.get('https://www.indeed.fr/emplois?q=développeur%2C+data+scientist%2C+data+analyst%2C+business+intelligence&l=' + str(city) + '&start=' + str(start))
            time.sleep(1)
            soup = BeautifulSoup(page.text,"lxml",from_encoding="utf-8")       
            for span in soup.find_all(name="span", attrs={"class":"salaryText"}): 
                salaries.append(span.text.strip())
    return(salaries)

#extract_salary_from_result(soup)


In [11]:
salary=extract_salary_from_result(soup)

In [12]:
def extract_summary_from_result(soup):
    summaries = [] 
   
    for city in cities:
        for start in range(0, max_results_per_city, 10):
            page=requests.get('https://www.indeed.fr/emplois?q=développeur%2C+data+scientist%2C+data+analyst%2C+business+intelligence&l=' + str(city) + '&start=' + str(start))
            time.sleep(1)
            soup = BeautifulSoup(page.text,"lxml",from_encoding="utf-8")       
            spans = soup.findAll('div', attrs={'class': 'summary'})
            for span in spans:
                summaries.append(span.text.strip())
    return(summaries)
#extract_summary_from_result(soup)

In [13]:
sommary=extract_summary_from_result(soup)

In [17]:
a=({'Titre': titre, 'Company': company, 'Location': location, 'Salary': salary, 'Summaries': sommary})
df= pd.DataFrame.from_dict(a, orient='index')
data=df.transpose()


In [19]:
data.to_csv("~/Desktop/data_indeed.csv" , sep=',', encoding='utf-8')

In [20]:
data.drop_duplicates(inplace=True) #dropping duplicates
data.Company.replace(regex=True,inplace=True,to_replace="\n",value="") #getting rid of /n in company
data.Salary.replace(regex=True, inplace=True, to_replace="\$", value="") #getting rid of $ in salary

In [28]:
data.isnull().sum() #shows total amount of null values for each column

Titre          31
Company         0
Location     1906
Salary       2818
Summaries       2
dtype: int64

In [29]:
data.dtypes

Titre        object
Company      object
Location     object
Salary       object
Summaries    object
dtype: object

In [30]:
data.shape 

(3578, 5)

In [31]:
data.describe()

,Titre,Company,Location,Salary,Summaries
count,3547,3578,1672,760,3576
unique,1291,855,37,137,1332
top,(ALTERNANCE) Développeur Frontend - Chez l’un ...,OpenClassrooms,France,2 210 € - 2 500 € par mois,"Euro Information, filiale Informatique du grou..."
freq,163,671,590,99,145


In [32]:
data.duplicated().sum()

0